In [15]:
import os
import cv2
import json
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
FILE_PATH = os.path.join(os.getcwd(), '../data/5_spins_with_mask/')
#Check if file exists
MASK_PATH = os.path.join(FILE_PATH,'masks')
if not os.path.exists(MASK_PATH):
    os.makedirs(MASK_PATH)

In [17]:
#Open transforms.json
with open(os.path.join(FILE_PATH,'transforms.json')) as f:
    transforms = json.load(f)

new_frames = []

cnt = 0
for f in transforms["frames"]:
    #Get Frame path
    frame_path = os.path.join(FILE_PATH,f["file_path"])
    #Open Frame
    frame = cv2.imread(frame_path)
    #Find edge
    edge = cv2.Canny(frame,100,200)
    edge_points = np.where(edge==255)
    #Get convex hull that contains edge
    hull = cv2.convexHull(np.array([edge_points[1],edge_points[0]]).T)
    #Create mask with hull
    mask = np.zeros((frame.shape[0],frame.shape[1]),dtype=np.uint8)
    cv2.fillConvexPoly(mask,hull,255)

    #Set alpha in frame to zero if mask is zero
    frame = np.concatenate((frame, np.expand_dims(mask, axis=2)), axis=2)

    #Save frame
    cv2.imwrite(frame_path,frame)

    #Save mask
    mask_path = os.path.join( "masks","{}.png".format(cnt))
    cv2.imwrite(os.path.join(FILE_PATH,mask_path),mask)

    f_copy = f.copy()
    f_copy["mask_path"] = mask_path
    new_frames.append(f_copy)

    cnt += 1
transforms["frames"] = new_frames

with open(os.path.join(FILE_PATH,'transforms.json'), 'w') as outfile:
    json.dump(transforms, outfile, indent=4)

(720, 1280, 3)
